# M2 : 230 - Figures and metrics

Evaluation scores on level-1 entities segmentation and classification with joint-labels method

In [47]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
    from google.colab import drive
    mountpoint = Path("/content/drive")
    drive.mount(str(mountpoint)) # Mount gdrive to BASE
    base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
    sys.path.append(str(base)) # Add BASE to Python Path
    BASE = Path(base).resolve() # Make BASE absolute
    DATASETS =  BASE / "dataset"
else:
    BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
    DATASETS = Path('/work/stual/dataset_ICDAR').resolve()
    OUT_BASE = Path('/work/stual/res_ICDAR').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR


## Constants

In [48]:
MAX_TRAINSET_SIZE = 6084

## Tools

In [49]:
import pandas as pd
import re
import os
import json

def compile_metrics(path): 
    rundirs = [f for f in os.listdir(path)]

    df = pd.DataFrame()
    for run_dir in rundirs:
        if 'run' in run_dir:
            run_path = path / run_dir
            nrun = re.search("\d+",run_dir)[0]

            files = [f for f in os.listdir(run_path) if "test_" in f and 'json' in f]
            sizes = [int(re.search("\d+",f)[0]) for f in files]

            for file, size in zip(files,sizes):
                file_path = run_path / file
                dftmp = pd.read_json(file_path, typ='series')
                dftmp = pd.DataFrame([dftmp])

                dftmp["trainsize"] = size 
                dftmp["run"] = nrun
                dftmp["trainsize_p"] = round(100 * size / MAX_TRAINSET_SIZE, 1)
                df = pd.concat([df, dftmp])

    return df.groupby(["run","trainsize"]).first()

# 231 - Experiment 1: tables on all-entities metrics

In [50]:
METRICS_DIR_REF = OUT_BASE / "method_2/m2-210-experiment_1_metrics"

In [51]:
# Load models metrics from metrics jsons
res = []
keys = []

if os.path.exists(METRICS_DIR_REF / "211-camembert-ner-joint-labelling-io/run_2"):
    camembert_ner_io_ref = compile_metrics(METRICS_DIR_REF / "211-camembert-ner-joint-labelling-io")
    camembert_ner_io_ref["eval_precision-l1l2"] = camembert_ner_io_ref["eval_precision"]
    camembert_ner_io_ref["eval_recall-l1l2"] = camembert_ner_io_ref["eval_recall"]
    camembert_ner_io_ref["eval_f1-l1l2"] = camembert_ner_io_ref["eval_f1"]
    camembert_ner_io_ref["eval_accuracy-l1l2"] = camembert_ner_io_ref["eval_accuracy"]
    res.append(camembert_ner_io_ref)
    keys.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR_REF / "212-camembert-ner-joint-labelling-iob2/run_2"):
    camembert_ner_iob2_ref = compile_metrics(METRICS_DIR_REF / "212-camembert-ner-joint-labelling-iob2")
    res.append(camembert_ner_iob2_ref)
    keys.append("CmBERT IOB2")

if os.path.exists(METRICS_DIR_REF / "213-pretrained-camembert-ner-joint-labelling-io/run_2"):
    ptrn_camembert_ner_io_ref = compile_metrics(METRICS_DIR_REF / "213-pretrained-camembert-ner-joint-labelling-io")
    ptrn_camembert_ner_io_ref["eval_precision-l1l2"] = ptrn_camembert_ner_io_ref["eval_precision"]
    ptrn_camembert_ner_io_ref["eval_recall-l1l2"] = ptrn_camembert_ner_io_ref["eval_recall"]
    ptrn_camembert_ner_io_ref["eval_f1-l1l2"] = ptrn_camembert_ner_io_ref["eval_f1"]
    ptrn_camembert_ner_io_ref["eval_accuracy-l1l2"] = ptrn_camembert_ner_io_ref["eval_accuracy"]
    res.append(ptrn_camembert_ner_io_ref)
    keys.append("Ptrn CmBERT IO")
    
if os.path.exists(METRICS_DIR_REF / "214-pretrained-camembert-ner-joint-labelling-iob2/run_2"):
    ptrn_camembert_ner_iob2_ref = compile_metrics(METRICS_DIR_REF / "214-pretrained-camembert-ner-joint-labelling-iob2")
    res.append(ptrn_camembert_ner_iob2_ref)
    keys.append("Ptrn CmBERT IOB2")
    
# Create the full table
print(keys)
metrics_raw_ref = pd.concat(res, keys=keys)
metrics_raw_ref

['CmBERT IO', 'CmBERT IOB2', 'Ptrn CmBERT IO', 'Ptrn CmBERT IOB2']


eval_loss  eval_precision  eval_recall  \
                 run trainsize                                           
CmBERT IO        1   6084        0.191235        0.959145     0.967932   
                 10  6084        0.160874        0.960657     0.965521   
                 11  6084        0.197560        0.961474     0.968776   
                 12  6084        0.228159        0.960794     0.969017   
                 13  6084        0.196876        0.954426     0.961905   
                 14  6084        0.212267        0.962235     0.961423   
                 15  6084        0.184041        0.957964     0.964316   
                 16  6084        0.184273        0.961290     0.966968   
                 17  6084        0.183668        0.958564     0.959373   
                 18  6084        0.173756        0.965805     0.963593   
                 19  6084        0.173339        0.957704     0.955395   
                 2   6084        0.219643        0.959671     0.969620   
                 20  6084        0.184615        0.962681     0.960940   
                 3   6084        0.216868        0.960798     0.969138   
                 4   6084        0.191439        0.955330     0.953948   
                 5   6084        0.163849        0.957006     0.963351   
                 6   6084        0.219385        0.947977     0.949005   
                 7   6084        0.218788        0.962954     0.968294   
                 8   6084        0.178479        0.951572     0.959373   
                 9   6084        0.171503        0.955122     0.962146   
CmBERT IOB2      1   6084        0.186733        0.955121     0.957768   
                 2   6084        0.218782        0.961185     0.960634   
                 3   6084        0.204611        0.963168     0.961972   
                 4   6084        0.233529        0.965537     0.958341   
                 5   6084        0.185404        0.959962     0.959870   
Ptrn CmBERT IO   1   6084        0.140634        0.950979     0.965883   
                 2   6084        0.175621        0.959498     0.968174   
                 3   6084        0.152491        0.956915     0.969259   
                 4   6084        0.148344        0.951454     0.966365   
                 5   6084        0.210482        0.958598     0.965763   
Ptrn CmBERT IOB2 1   6084        0.190223        0.960122     0.963883   
                 2   6084        0.209766        0.960595     0.961972   
                 3   6084        0.173063        0.954886     0.962641   
                 4   6084        0.173732        0.954429     0.962545   
                 5   6084        0.213284        0.958444     0.967418   

                                 eval_f1  eval_accuracy  eval_precision-l1  \
                 run trainsize                                               
CmBERT IO        1   6084       0.963519       0.942002           0.957034   
                 10  6084       0.963083       0.949381           0.954043   
                 11  6084       0.965111       0.948452           0.960035   
                 12  6084       0.964888       0.947265           0.962890   
                 13  6084       0.958151       0.940248           0.957484   
                 14  6084       0.961828       0.935965           0.939446   
                 15  6084       0.961129       0.944530           0.958425   
                 16  6084       0.964120       0.947162           0.951427   
                 17  6084       0.958968       0.939009           0.941342   
                 18  6084       0.964697       0.945150           0.962418   
                 19  6084       0.956548       0.938700           0.931862   
                 2   6084       0.964620       0.948091           0.957345   
                 20  6084       0.961810       0.937255           0.946799   
                 3   6084       0.964950       0.944221           0.961530   
                 4   6084       0.954639     

In [52]:
metrics_raw_ref.columns

Index(['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1',
       'eval_accuracy', 'eval_precision-l1', 'eval_recall-l1', 'eval_f1-l1',
       'eval_accuracy-l1', 'eval_precision-l2', 'eval_recall-l2', 'eval_f1-l2',
       'eval_accuracy-l2', 'eval_precision-all', 'eval_recall-all',
       'eval_f1-all', 'eval_accuracy-all', 'eval_precision-das',
       'eval_recall-das', 'eval_f1-das', 'eval_accuracy-das', 'eval_PER',
       'eval_ACT', 'eval_ACT_L1', 'eval_ACT_L2', 'eval_DESC', 'eval_TITREH',
       'eval_TITREP', 'eval_SPAT', 'eval_LOC', 'eval_CARDINAL', 'eval_FT',
       'eval_TITRE', 'eval_runtime', 'eval_samples_per_second',
       'eval_steps_per_second', 'epoch', 'trainsize_p', 'eval_precision-l1l2',
       'eval_recall-l1l2', 'eval_f1-l1l2', 'eval_accuracy-l1l2'],
      dtype='object')

## 231.1 Build the averaged table


In [53]:
# Store p/r/f1 as percentages
eval_ = ["eval_f1","eval_f1-l1","eval_f1-all","eval_f1-l2",'eval_f1-l1l2','eval_f1-das']
metrics_ref = metrics_raw_ref.copy()
metrics_ref[eval_] = metrics_raw_ref[eval_].multiply(100., axis=1)
metrics_ref.head()

eval_loss  eval_precision  eval_recall    eval_f1  \
          run trainsize                                                      
CmBERT IO 1   6084        0.191235        0.959145     0.967932  96.351854   
          10  6084        0.160874        0.960657     0.965521  96.308321   
          11  6084        0.197560        0.961474     0.968776  96.511139   
          12  6084        0.228159        0.960794     0.969017  96.488806   
          13  6084        0.196876        0.954426     0.961905  95.815071   

                         eval_accuracy  eval_precision-l1  eval_recall-l1  \
          run trainsize                                                     
CmBERT IO 1   6084            0.942002           0.957034        0.966094   
          10  6084            0.949381           0.954043        0.968956   
          11  6084            0.948452           0.960035        0.967856   
          12  6084            0.947265           0.962890        0.965434   
          13  6084            0.940248           0.957484        0.961911   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  ...  \
          run trainsize                                                   ...   
CmBERT IO 1   6084        96.154268          0.965067           0.961497  ...   
          10  6084        96.144184          0.967699           0.966477  ...   
          11  6084        96.392939          0.966099           0.966980  ...   
          12  6084        96.416007          0.970175           0.963971  ...   
          13  6084        95.969248          0.962332           0.959091  ...   

                         eval_TITRE  eval_runtime  eval_samples_per_second  \
          run trainsize                                                      
CmBERT IO 1   6084             None        9.0450                  186.291   
          10  6084             None        8.8620                  190.138   
          11  6084             None        9.3453                  180.305   
          12  6084             None       10.4733                  160.886   
          13  6084             None       10.7838                  156.252   

                         eval_steps_per_second  epoch  trainsize_p  \
          run trainsize                                              
CmBERT IO 1   6084                      11.719   6.30        100.0   
          10  6084                      11.961   4.72        100.0   
          11  6084                      11.343   6.82        100.0   
          12  6084                      10.121  10.24        100.0   
          13  6084                       9.830   4.99        100.0   

                         eval_precision-l1l2  eval_recall-l1l2  eval_f1-l1l2  \
          run trainsize                                                        
CmBERT IO 1   6084                  0.959145          0.967932     96.351854   
          10  6084                  0.960657          0.965521     96.308321   
          11  6084                  0.961474          0.968776     96.511139   
          12  6084                  0.960794          0.969017     96.488806   
          13  6084                  0.954426          0.961905     95.815071   

                         eval_accuracy-l1l2  
          run trainsize                      
CmBERT IO 1   6084                 0.942002  
          10  6084                 0.949381  
          11  6084                 0.948452  
          12  6084                 0.947265  
          13  6084                 0.940248  

[5 rows x 42 columns]

In [54]:
# Average over runs
averaged_ref = metrics_ref.groupby(level=0).apply(lambda grp: grp.groupby(by="trainsize").mean())
averaged_ref.set_index(["trainsize_p"], append=True,inplace=True)

# Keep just the necessary columns
averaged_ref=averaged_ref[["eval_f1",'eval_f1-l1l2',"eval_f1-all","eval_f1-l1","eval_f1-l2",'eval_f1-das']]

# Set pretty names
averaged_ref.index.names = ['Model','Trainset Size',"%"]
averaged_ref.rename(columns={"eval_f1":"P+L1+P+L2 (train)",
                             "eval_f1-l1l2":"L1+L2",
                             'eval_f1-all':"All",
                            "eval_f1-l1":"Level 1",
                            "eval_f1-l2":"Level 2",
                            'eval_f1-das':"DAS alignement"
                         }, errors="raise", inplace=True)
averaged_ref.rename(mapper={"camembert_io_ref": "CmBERT IO","camembert_iob2_ref": "CmBERT IOB2","prtn_camembert_io_ref": "Ptrn CmBERT IO","prtn_camembert_iob2_ref": "Ptrn CmBERT IOB2"}, errors="ignore", inplace=True, axis=0)
averaged_ref

,,,P+L1+P+L2 (train),L1+L2,All,Level 1,Level 2,DAS alignement
Model,Trainset Size,%,,,,,,
CmBERT IO,6084,100.0,96.082082,96.082082,96.124431,95.488981,96.916017,96.630067
CmBERT IOB2,6084,100.0,96.035288,96.030419,95.982694,95.291741,96.839731,96.737652
Ptrn CmBERT IO,6084,100.0,96.125104,96.125104,96.289239,95.970882,96.682303,96.877739
Ptrn CmBERT IOB2,6084,100.0,96.068198,96.035353,96.077814,95.846368,96.363276,96.912063


## 231.2 Create the results table

In [55]:
latex_table_ref = averaged_ref.copy()

caption = "F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on reference dataset with Independent Flat NER layers approach (M1)."
print(latex_table_ref.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table_ref

\begin{table}
\centering
\caption{F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on reference dataset with Independent Flat NER layers approach (M1).}
\begin{tabular}{lllrrrrrr}
\toprule
                 &      &       &  P+L1+P+L2 (train) &  L1+L2 &  All &  Level 1 &  Level 2 &  DAS alignement \\
Model & Trainset Size & \% &                    &        &      &          &          &                 \\
\midrule
CmBERT IO & 6084 & 100.0 &               96.1 &   96.1 & 96.1 &     95.5 &     96.9 &            96.6 \\
CmBERT IOB2 & 6084 & 100.0 &               96.0 &   96.0 & 96.0 &     95.3 &     96.8 &            96.7 \\
Ptrn CmBERT IO & 6084 & 100.0 &               96.1 &   96.1 & 96.3 &     96.0 &     96.7 &            96.9 \\
Ptrn CmBERT IOB2 & 6084 & 100.0 &               96.1 &   96.0 & 96.1 &     95.8 &     96.4 &            96.9 \\
\bottomrule
\end{tabular}
\end{table}



,,,P+L1+P+L2 (train),L1+L2,All,Level 1,Level 2,DAS alignement
Model,Trainset Size,%,,,,,,
CmBERT IO,6084,100.0,96.082082,96.082082,96.124431,95.488981,96.916017,96.630067
CmBERT IOB2,6084,100.0,96.035288,96.030419,95.982694,95.291741,96.839731,96.737652
Ptrn CmBERT IO,6084,100.0,96.125104,96.125104,96.289239,95.970882,96.682303,96.877739
Ptrn CmBERT IOB2,6084,100.0,96.068198,96.035353,96.077814,95.846368,96.363276,96.912063


# 232 - Experiment 2: tables

In [56]:
METRICS_DIR_PERO = OUT_BASE / "method_2/m2-220-experiment_2_metrics"

In [57]:
# Load models metrics from metrics jsons
res = []
keys = []

if os.path.exists(METRICS_DIR_PERO / "221-camembert-ner-joint-labelling-io/run_2"):
    camembert_ner_io_pero = compile_metrics(METRICS_DIR_PERO / "221-camembert-ner-joint-labelling-io")
    camembert_ner_io_pero["eval_precision-l1l2"] = camembert_ner_io_pero["eval_precision"]
    camembert_ner_io_pero["eval_recall-l1l2"] = camembert_ner_io_pero["eval_recall"]
    camembert_ner_io_pero["eval_f1-l1l2"] = camembert_ner_io_pero["eval_f1"]
    camembert_ner_io_pero["eval_accuracy-l1l2"] = camembert_ner_io_pero["eval_accuracy"]
    res.append(camembert_ner_io_pero)
    keys.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR_PERO / "222-camembert-ner-joint-labelling-iob2/run_2"):
    camembert_ner_iob2_pero = compile_metrics(METRICS_DIR_PERO / "222-camembert-ner-joint-labelling-iob2")
    res.append(camembert_ner_iob2_pero)
    keys.append("CmBERT IOB2")

if os.path.exists(METRICS_DIR_PERO / "223-pretrained-camembert-ner-joint-labelling-io/run_2"):
    ptrn_camembert_ner_io_pero = compile_metrics(METRICS_DIR_PERO / "223-pretrained-camembert-ner-joint-labelling-io")
    ptrn_camembert_ner_io_pero["eval_precision-l1l2"] = ptrn_camembert_ner_io_pero["eval_precision"]
    ptrn_camembert_ner_io_pero["eval_recall-l1l2"] = ptrn_camembert_ner_io_pero["eval_recall"]
    ptrn_camembert_ner_io_pero["eval_f1-l1l2"] = ptrn_camembert_ner_io_pero["eval_f1"]
    ptrn_camembert_ner_io_pero["eval_accuracy-l1l2"] = ptrn_camembert_ner_io_pero["eval_accuracy"]
    res.append(ptrn_camembert_ner_io_pero)
    keys.append("Ptrn CmBERT IO")
    
if os.path.exists(METRICS_DIR_PERO / "224-pretrained-camembert-ner-joint-labelling-iob2/run_2"):
    ptrn_camembert_ner_iob2_pero = compile_metrics(METRICS_DIR_PERO / "224-pretrained-camembert-ner-joint-labelling-iob2")
    res.append(ptrn_camembert_ner_iob2_pero)
    keys.append("Ptrn CmBERT IOB2")
    
# Create the full table
print(keys)
metrics_raw_pero = pd.concat(res, keys=keys)
metrics_raw_pero

['CmBERT IO', 'CmBERT IOB2', 'Ptrn CmBERT IO', 'Ptrn CmBERT IOB2']


eval_loss  eval_precision  eval_recall  \
                 run trainsize                                           
CmBERT IO        1   6084        0.291394        0.940070     0.948253   
                 10  6084        0.256732        0.939662     0.948978   
                 11  6084        0.280147        0.937718     0.941120   
                 12  6084        0.228882        0.938800     0.949583   
                 13  6084        0.221750        0.930985     0.942691   
                 14  6084        0.278953        0.938272     0.946439   
                 15  6084        0.219882        0.929524     0.944021   
                 16  6084        0.229505        0.945308     0.948737   
                 17  6084        0.232668        0.930584     0.938460   
                 18  6084        0.258292        0.936753     0.947286   
                 19  6084        0.226856        0.924096     0.936162   
                 2   6084        0.254665        0.942836     0.945230   
                 20  6084        0.242449        0.936130     0.949825   
                 3   6084        0.209478        0.940993     0.946681   
                 4   6084        0.240886        0.924360     0.938218   
                 5   6084        0.227982        0.938368     0.948011   
                 6   6084        0.203236        0.942298     0.943779   
                 7   6084        0.285654        0.943414     0.947407   
                 8   6084        0.241357        0.930833     0.942087   
                 9   6084        0.245520        0.942642     0.951759   
CmBERT IOB2      1   6084        0.286570        0.943665     0.945380   
                 2   6084        0.298264        0.941362     0.942893   
                 3   6084        0.274712        0.933964     0.944328   
                 4   6084        0.352424        0.943673     0.948728   
                 5   6084        0.268743        0.945472     0.938779   
Ptrn CmBERT IO   1   6084        0.200325        0.935673     0.947890   
                 2   6084        0.218722        0.927437     0.945714   
                 3   6084        0.232018        0.929791     0.947890   
                 4   6084        0.201813        0.942492     0.953089   
                 5   6084        0.211277        0.932397     0.947165   
Ptrn CmBERT IOB2 1   6084        0.298362        0.942547     0.947867   
                 2   6084        0.228853        0.938248     0.944710   
                 3   6084        0.225670        0.935934     0.943275   
                 4   6084        0.278106        0.938586     0.945858   
                 5   6084        0.250595        0.934945     0.947197   

                                 eval_f1  eval_accuracy  eval_precision-l1  \
                 run trainsize                                               
CmBERT IO        1   6084       0.944143       0.937151           0.932435   
                 10  6084       0.944297       0.929269           0.917380   
                 11  6084       0.939416       0.925876           0.927165   
                 12  6084       0.944161       0.937882           0.931311   
                 13  6084       0.936802       0.928747           0.915247   
                 14  6084       0.942338       0.925406           0.925806   
                 15  6084       0.936716       0.925928           0.915898   
                 16  6084       0.947019       0.933497           0.921801   
                 17  6084       0.934505       0.924101           0.911164   
                 18  6084       0.941990       0.930208           0.927687   
                 19  6084       0.930090       0.921595           0.895847   
                 2   6084       0.944032       0.928642           0.926398   
                 20  6084       0.942927       0.931826           0.926302   
                 3   6084       0.943828       0.935272           0.924157   
                 4   6084       0.931237     

In [58]:
metrics_raw_pero.columns

Index(['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1',
       'eval_accuracy', 'eval_precision-l1', 'eval_recall-l1', 'eval_f1-l1',
       'eval_accuracy-l1', 'eval_precision-l2', 'eval_recall-l2', 'eval_f1-l2',
       'eval_accuracy-l2', 'eval_precision-all', 'eval_recall-all',
       'eval_f1-all', 'eval_accuracy-all', 'eval_precision-das',
       'eval_recall-das', 'eval_f1-das', 'eval_accuracy-das', 'eval_PER',
       'eval_ACT', 'eval_ACT_L1', 'eval_ACT_L2', 'eval_DESC', 'eval_TITREH',
       'eval_TITREP', 'eval_SPAT', 'eval_LOC', 'eval_CARDINAL', 'eval_FT',
       'eval_TITRE', 'eval_runtime', 'eval_samples_per_second',
       'eval_steps_per_second', 'epoch', 'trainsize_p', 'eval_precision-l1l2',
       'eval_recall-l1l2', 'eval_f1-l1l2', 'eval_accuracy-l1l2'],
      dtype='object')

In [59]:
# Store p/r/f1 as percentages
eval_ = ["eval_f1","eval_f1-l1","eval_f1-all","eval_f1-l2",'eval_f1-l1l2','eval_f1-das']
metrics_pero = metrics_raw_pero.copy()
metrics_pero[eval_] = metrics_raw_pero[eval_].multiply(100., axis=1)
metrics_pero.head()

eval_loss  eval_precision  eval_recall    eval_f1  \
          run trainsize                                                      
CmBERT IO 1   6084        0.291394        0.940070     0.948253  94.414349   
          10  6084        0.256732        0.939662     0.948978  94.429740   
          11  6084        0.280147        0.937718     0.941120  93.941588   
          12  6084        0.228882        0.938800     0.949583  94.416061   
          13  6084        0.221750        0.930985     0.942691  93.680163   

                         eval_accuracy  eval_precision-l1  eval_recall-l1  \
          run trainsize                                                     
CmBERT IO 1   6084            0.937151           0.932435        0.944958   
          10  6084            0.929269           0.917380        0.943637   
          11  6084            0.925876           0.927165        0.933289   
          12  6084            0.937882           0.931311        0.946279   
          13  6084            0.928747           0.915247        0.934390   

                         eval_f1-l1  eval_accuracy-l1  eval_precision-l2  ...  \
          run trainsize                                                   ...   
CmBERT IO 1   6084        93.865500          0.961215           0.944160  ...   
          10  6084        93.032342          0.950984           0.942519  ...   
          11  6084        93.021725          0.950201           0.945391  ...   
          12  6084        93.873539          0.964817           0.943966  ...   
          13  6084        92.471947          0.946912           0.935190  ...   

                                                                eval_TITRE  \
          run trainsize                                                      
CmBERT IO 1   6084       {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, '...   
          10  6084                                                    None   
          11  6084                                                    None   
          12  6084                                                    None   
          13  6084                                                    None   

                         eval_runtime  eval_samples_per_second  \
          run trainsize                                          
CmBERT IO 1   6084            10.5194                  160.181   
          10  6084            10.8105                  155.867   
          11  6084            10.5991                  158.976   
          12  6084            10.8705                  155.007   
          13  6084             9.7219                  173.320   

                         eval_steps_per_second  epoch  trainsize_p  \
          run trainsize                                              
CmBERT IO 1   6084                      10.077   8.92        100.0   
          10  6084                       9.805   6.30        100.0   
          11  6084                      10.001   5.77        100.0   
          12  6084                       9.751   6.56        100.0   
          13  6084                      10.903   3.94        100.0   

                         eval_precision-l1l2  eval_recall-l1l2  eval_f1-l1l2  \
          run trainsize                                                        
CmBERT IO 1   6084                  0.940070          0.948253     94.414349   
          10  6084                  0.939662          0.948978     94.429740   
          11  6084                  0.937718          0.941120     93.941588   
          12  6084                  0.938800          0.949583     94.416061   
          13  6084                  0.930985          0.942691     93.680163   

                         eval_accuracy-l1l2  
          run trainsize                      
CmBERT IO 1   6084                 0.937151  
          10  6084                 0.929269  
          11  6084                 0.925876  
          12  6084                 0.937882  
          13  6084              

In [60]:
# Average over runs
averaged_pero = metrics_pero.groupby(level=0).apply(lambda grp: grp.groupby(by="trainsize").mean())
averaged_pero.set_index(["trainsize_p"], append=True,inplace=True)

# Keep just the necessary columns
averaged_pero=averaged_pero[["eval_f1",'eval_f1-l1l2',"eval_f1-all","eval_f1-l1","eval_f1-l2",'eval_f1-das']]

# Set pretty names
averaged_pero.index.names = ['Model','Trainset Size',"%"]
averaged_pero.rename(columns={"eval_f1":"P+L1+P+L2 (train)",
                             "eval_f1-l1l2":"L1+L2",
                              'eval_f1-all':"All",
                            "eval_f1-l1":"Level 1",
                            "eval_f1-l2":"Level 2",
                            'eval_f1-das':"DAS alignement"
                         }, errors="raise", inplace=True)
averaged_pero.rename(mapper={"camembert_io_pero": "CmBERT IO","camembert_iob2_pero": "CmBERT IOB2","prtn_camembert_io_pero": "Ptrn CmBERT IO","prtn_camembert_iob2_pero": "Ptrn CmBERT IOB2"}, errors="ignore", inplace=True, axis=0)
averaged_pero

,,,P+L1+P+L2 (train),L1+L2,All,Level 1,Level 2,DAS alignement
Model,Trainset Size,%,,,,,,
CmBERT IO,6084,100.0,94.093580,94.093580,93.695383,93.101579,94.431895,94.549476
CmBERT IOB2,6084,100.0,94.281452,94.161059,93.771105,93.195417,94.486534,94.690020
Ptrn CmBERT IO,6084,100.0,94.089300,94.089300,93.882361,93.437655,94.432368,94.612859
Ptrn CmBERT IOB2,6084,100.0,94.189938,94.119129,93.722318,93.064214,94.535459,94.847359


In [61]:
latex_table_pero = averaged_pero.copy()

caption = "F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on noisy dataset with Joint-labelling approach (M2)."
print(latex_table_pero.to_latex(float_format="%.1f", multirow=True, caption=caption))
latex_table_pero

\begin{table}
\centering
\caption{F1 score measured on the fine-tuned models CmBERT, CmBERT+ptrn on noisy dataset with Joint-labelling approach (M2).}
\begin{tabular}{lllrrrrrr}
\toprule
                 &      &       &  P+L1+P+L2 (train) &  L1+L2 &  All &  Level 1 &  Level 2 &  DAS alignement \\
Model & Trainset Size & \% &                    &        &      &          &          &                 \\
\midrule
CmBERT IO & 6084 & 100.0 &               94.1 &   94.1 & 93.7 &     93.1 &     94.4 &            94.5 \\
CmBERT IOB2 & 6084 & 100.0 &               94.3 &   94.2 & 93.8 &     93.2 &     94.5 &            94.7 \\
Ptrn CmBERT IO & 6084 & 100.0 &               94.1 &   94.1 & 93.9 &     93.4 &     94.4 &            94.6 \\
Ptrn CmBERT IOB2 & 6084 & 100.0 &               94.2 &   94.1 & 93.7 &     93.1 &     94.5 &            94.8 \\
\bottomrule
\end{tabular}
\end{table}



,,,P+L1+P+L2 (train),L1+L2,All,Level 1,Level 2,DAS alignement
Model,Trainset Size,%,,,,,,
CmBERT IO,6084,100.0,94.093580,94.093580,93.695383,93.101579,94.431895,94.549476
CmBERT IOB2,6084,100.0,94.281452,94.161059,93.771105,93.195417,94.486534,94.690020
Ptrn CmBERT IO,6084,100.0,94.089300,94.089300,93.882361,93.437655,94.432368,94.612859
Ptrn CmBERT IOB2,6084,100.0,94.189938,94.119129,93.722318,93.064214,94.535459,94.847359


# 233 - Experiments 1 & 2: table

Build averaged table of F1-score for each dataset, each BERT-based model and each annotation model.

In [62]:
averaged = pd.concat([averaged_ref,averaged_pero],keys=["Reference","Pero OCR"])
averaged = averaged.reset_index(level=[2,3], drop=True)
averaged

P+L1+P+L2 (train)      L1+L2        All  \
          Model                                                       
Reference CmBERT IO                 96.082082  96.082082  96.124431   
          CmBERT IOB2               96.035288  96.030419  95.982694   
          Ptrn CmBERT IO            96.125104  96.125104  96.289239   
          Ptrn CmBERT IOB2          96.068198  96.035353  96.077814   
Pero OCR  CmBERT IO                 94.093580  94.093580  93.695383   
          CmBERT IOB2               94.281452  94.161059  93.771105   
          Ptrn CmBERT IO            94.089300  94.089300  93.882361   
          Ptrn CmBERT IOB2          94.189938  94.119129  93.722318   

                              Level 1    Level 2  DAS alignement  
          Model                                                   
Reference CmBERT IO         95.488981  96.916017       96.630067  
          CmBERT IOB2       95.291741  96.839731       96.737652  
          Ptrn CmBERT IO    95.970882  96.682303       96.877739  
          Ptrn CmBERT IOB2  95.846368  96.363276       96.912063  
Pero OCR  CmBERT IO         93.101579  94.431895       94.549476  
          CmBERT IOB2       93.195417  94.486534       94.690020  
          Ptrn CmBERT IO    93.437655  94.432368       94.612859  
          Ptrn CmBERT IOB2  93.064214  94.535459       94.847359

In [63]:
caption = "F1 score measured on the fine-tuned models CmBERTand CmBERT+ptrn on reference dataset and noisy dataset with Joint-labelling approach (M2)."
print(averaged.to_latex(float_format="%.1f", multirow=True, caption=caption))
averaged

\begin{table}
\centering
\caption{F1 score measured on the fine-tuned models CmBERTand CmBERT+ptrn on reference dataset and noisy dataset with Joint-labelling approach (M2).}
\begin{tabular}{llrrrrrr}
\toprule
         &                  &  P+L1+P+L2 (train) &  L1+L2 &  All &  Level 1 &  Level 2 &  DAS alignement \\
{} & Model &                    &        &      &          &          &                 \\
\midrule
\multirow{4}{*}{Reference} & CmBERT IO &               96.1 &   96.1 & 96.1 &     95.5 &     96.9 &            96.6 \\
         & CmBERT IOB2 &               96.0 &   96.0 & 96.0 &     95.3 &     96.8 &            96.7 \\
         & Ptrn CmBERT IO &               96.1 &   96.1 & 96.3 &     96.0 &     96.7 &            96.9 \\
         & Ptrn CmBERT IOB2 &               96.1 &   96.0 & 96.1 &     95.8 &     96.4 &            96.9 \\
\cline{1-8}
\multirow{4}{*}{Pero OCR} & CmBERT IO &               94.1 &   94.1 & 93.7 &     93.1 &     94.4 &            94.5 \\
         & CmBER

P+L1+P+L2 (train)      L1+L2        All  \
          Model                                                       
Reference CmBERT IO                 96.082082  96.082082  96.124431   
          CmBERT IOB2               96.035288  96.030419  95.982694   
          Ptrn CmBERT IO            96.125104  96.125104  96.289239   
          Ptrn CmBERT IOB2          96.068198  96.035353  96.077814   
Pero OCR  CmBERT IO                 94.093580  94.093580  93.695383   
          CmBERT IOB2               94.281452  94.161059  93.771105   
          Ptrn CmBERT IO            94.089300  94.089300  93.882361   
          Ptrn CmBERT IOB2          94.189938  94.119129  93.722318   

                              Level 1    Level 2  DAS alignement  
          Model                                                   
Reference CmBERT IO         95.488981  96.916017       96.630067  
          CmBERT IOB2       95.291741  96.839731       96.737652  
          Ptrn CmBERT IO    95.970882  96.682303       96.877739  
          Ptrn CmBERT IOB2  95.846368  96.363276       96.912063  
Pero OCR  CmBERT IO         93.101579  94.431895       94.549476  
          CmBERT IOB2       93.195417  94.486534       94.690020  
          Ptrn CmBERT IO    93.437655  94.432368       94.612859  
          Ptrn CmBERT IOB2  93.064214  94.535459       94.847359

# 234 : Experiments 1 and 2: table by classe

In [46]:
import pandas as pd
import json
from pandas import json_normalize

def compile_metrics_by_classes(path, classes): 
    rundirs = [f for f in os.listdir(path)]

    df = pd.DataFrame()
    for run_dir in rundirs:
        if 'run' in run_dir:
            run_path = path / run_dir
            nrun = re.search("\d+",run_dir)[0]

            files = [f for f in os.listdir(run_path) if "test_" in f and 'json' in f]
            sizes = [int(re.search("\d+",f)[0]) for f in files]
                
            for file, size in zip(files,sizes):
                file_path = run_path / file
                dftmp = pd.read_json(file_path)
                classes_dict = {key: dftmp[key] for key in classes}
                dftmp = pd.DataFrame.from_dict(classes_dict)
                dftmp = dftmp.T
                dftmp['number'] = dftmp['number'].astype(int)
                dftmp["trainsize"] = size 
                dftmp["run"] = nrun
                dftmp["trainsize_p"] = round(100 * size / MAX_TRAINSET_SIZE, 1)
                df = pd.concat([df, dftmp])
                df["classe"] = df.index
                
    return df.groupby(["run","classe"]).first()

In [11]:
classes = ['eval_PER','eval_ACT','eval_ACT_L1','eval_ACT_L2','eval_DESC','eval_TITREH','eval_TITREP','eval_SPAT','eval_LOC','eval_CARDINAL','eval_FT'
 #'eval_TITRE'
]

# Load models metrics from metrics jsons
res = []
keys = []

if os.path.exists(METRICS_DIR / "211-camembert-ner-joint-labelling-io/run_2"):
    camembert_ner_io = compile_metrics_by_classes(METRICS_DIR / "211-camembert-ner-joint-labelling-io",classes)
    res.append(camembert_ner_io)
    keys.append("CmBERT IO")
    
if os.path.exists(METRICS_DIR / "212-camembert-ner-joint-labelling-iob2/run_2"):
    camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR / "212-camembert-ner-joint-labelling-iob2",classes)
    res.append(camembert_ner_iob2)
    keys.append("CmBERT IOB2")
    
if os.path.exists(METRICS_DIR / "213-pretrained-camembert-ner-joint-labelling-io-classes/run_2"):
    ptrn_camembert_ner_io = compile_metrics_by_classes(METRICS_DIR / "213-pretrained-camembert-ner-joint-labelling-io-classes",classes)
    res.append(ptrn_camembert_ner_io)
    keys.append("Ptrn CmBERT IO")
    
"""
if os.path.exists(METRICS_DIR / "214-pretrained-camembert-ner-joint-labelling-iob2/run_2"):
    ptrn_camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR / "214-pretrained-camembert-ner-joint-labelling-iob2-classes",classes)
    res.append(ptrn_camembert_ner_iob2)
    keys.append("Ptrn CmBERT IOB2")"""

'\nif os.path.exists(METRICS_DIR / "214-pretrained-camembert-ner-joint-labelling-iob2/run_2"):\n    ptrn_camembert_ner_iob2 = compile_metrics_by_classes(METRICS_DIR / "214-pretrained-camembert-ner-joint-labelling-iob2-classes",classes)\n    res.append(ptrn_camembert_ner_iob2)\n    keys.append("Ptrn CmBERT IOB2")'

In [12]:
metrics_raw_classes = pd.concat(res, keys=keys)
metrics_raw_classes

precision    recall        f1  number  \
               run classe                                                 
CmBERT IO      1   eval_ACT        0.894783  0.925046  0.909663    1094   
                   eval_ACT_L1     0.953488  0.954413  0.953951    1031   
                   eval_ACT_L2     0.282828  0.444444  0.345679      63   
                   eval_CARDINAL   0.997719  0.999429  0.998573    1751   
                   eval_DESC       0.524752  0.670886  0.588889      79   
...                                     ...       ...       ...     ...   
Ptrn CmBERT IO 5   eval_LOC        0.968316  0.974818  0.971556    1787   
                   eval_PER        0.976923  0.979822  0.978370    1685   
                   eval_SPAT       0.976027  0.978821  0.977422    1747   
                   eval_TITREH     1.000000  1.000000  1.000000      43   
                   eval_TITREP     0.576923  0.454545  0.508475      33   

                                  trainsize  trainsize_p  
               run classe                                 
CmBERT IO      1   eval_ACT            6084        100.0  
                   eval_ACT_L1         6084        100.0  
                   eval_ACT_L2         6084        100.0  
                   eval_CARDINAL       6084        100.0  
                   eval_DESC           6084        100.0  
...                                     ...          ...  
Ptrn CmBERT IO 5   eval_LOC            6084        100.0  
                   eval_PER            6084        100.0  
                   eval_SPAT           6084        100.0  
                   eval_TITREH         6084        100.0  
                   eval_TITREP         6084        100.0  

[330 rows x 6 columns]

In [13]:
# Store p/r/f1 as percentages
val = ["f1","precision","recall"]
metrics_raw_classes = metrics_raw_classes.copy()
metrics_raw_classes[val] = metrics_raw_classes[val].multiply(100., axis=1)
metrics_raw_classes.head()
metrics_raw_classes

precision      recall          f1  number  \
               run classe                                                      
CmBERT IO      1   eval_ACT        89.478338   92.504570   90.966292    1094   
                   eval_ACT_L1     95.348837   95.441319   95.395056    1031   
                   eval_ACT_L2     28.282828   44.444444   34.567901      63   
                   eval_CARDINAL   99.771950   99.942890   99.857347    1751   
                   eval_DESC       52.475248   67.088608   58.888889      79   
...                                      ...         ...         ...     ...   
Ptrn CmBERT IO 5   eval_LOC        96.831573   97.481813   97.155605    1787   
                   eval_PER        97.692308   97.982196   97.837037    1685   
                   eval_SPAT       97.602740   97.882084   97.742212    1747   
                   eval_TITREH    100.000000  100.000000  100.000000      43   
                   eval_TITREP     57.692308   45.454545   50.847458      33   

                                  trainsize  trainsize_p  
               run classe                                 
CmBERT IO      1   eval_ACT            6084        100.0  
                   eval_ACT_L1         6084        100.0  
                   eval_ACT_L2         6084        100.0  
                   eval_CARDINAL       6084        100.0  
                   eval_DESC           6084        100.0  
...                                     ...          ...  
Ptrn CmBERT IO 5   eval_LOC            6084        100.0  
                   eval_PER            6084        100.0  
                   eval_SPAT           6084        100.0  
                   eval_TITREH         6084        100.0  
                   eval_TITREP         6084        100.0  

[330 rows x 6 columns]

In [14]:
# Average over runs
averaged_classes = metrics_raw_classes.groupby(level=0).apply(lambda grp: grp.groupby(by="classe").mean())
averaged_classes.set_index(["number"],
                   append=True,
                   inplace=True)

# Keep just the necessary columns
averaged_classes=averaged_classes[["precision","recall","f1"]]

# Set pretty names
averaged_classes.index.names = ['Test','Entity type',"Count"]
averaged_classes.rename(mapper={'eval_PER':'PER','eval_ACT':'ACT','eval_ACT_L1':'ACT L1 only','eval_ACT_L2':'ACT L2 only','eval_DESC':'DESC','eval_TITREH':'TITREH','eval_TITREP':'TITREP','eval_SPAT':'SPAT','eval_LOC':'LOC','eval_CARDINAL':'CARDINAL','eval_FT':'FT'}, errors="ignore", inplace=True, axis=0)
averaged_classes

precision     recall         f1
Test           Entity type Count                                  
CmBERT IO      ACT         1094.0  89.740092  91.654479  90.682136
               ACT L1 only 1031.0  93.545559  94.975752  94.228502
               ACT L2 only 63.0    29.614605  37.301587  31.652003
               CARDINAL    1751.0  99.700620  99.822958  99.761700
               DESC        79.0    37.827240  48.924051  42.381466
               FT          14.0    52.050866  31.071429  35.933656
               LOC         1787.0  97.176720  97.714046  97.444382
               PER         1685.0  96.860526  97.516320  97.185476
               SPAT        1747.0  96.951020  97.681740  97.314619
               TITREH      43.0    94.534345  97.441860  95.801115
               TITREP      33.0    57.887927  46.666667  51.279550
CmBERT IOB2    ACT         1094.0  89.196356  91.517367  90.338967
               ACT L1 only 1031.0  92.719293  94.878758  93.783417
               ACT L2 only 63.0    34.967322  36.507937  35.344414
               CARDINAL    1751.0  99.612392  99.782981  99.697562
               DESC        79.0    31.737330  44.050633  36.810314
               FT          14.0    59.428571  32.857143  39.806076
               LOC         1788.0  96.939910  97.729306  97.332429
               PER         1685.0  96.995620  97.709199  97.350988
               SPAT        1747.0  97.035938  97.721809  97.377046
               TITREH      43.0    97.683983  98.139535  97.898580
               TITREP      33.0    60.042875  53.333333  55.978269
Ptrn CmBERT IO ACT         1094.0  88.288148  92.029250  90.115511
               ACT L1 only 1031.0  94.013171  95.227934  94.613351
               ACT L2 only 63.0    26.592664  39.682540  31.622461
               CARDINAL    1751.0  99.612125  99.782981  99.697425
               DESC        79.0    41.151738  57.974684  47.674517
               FT          14.0    66.138889  52.857143  58.148272
               LOC         1787.0  97.232347  97.895915  97.562956
               PER         1685.0  97.980483  98.468843  98.223989
               SPAT        1747.0  96.831613  97.824843  97.324784
               TITREH      43.0    98.150106  98.604651  98.374766
               TITREP      33.0    59.222985  55.757576  57.300857